In [ ]:
import numpy as np
from pydrake.all import (
    StartMeshcat,
    DirectCollocation,
    DiagramBuilder,
    AddMultibodyPlantSceneGraph,
    Parser,
    JointIndex,
    RevoluteJoint,
    PassThrough,
    LogVectorOutput,
    MeshcatVisualizer,
    Simulator,
    Solve,
    SnoptSolver,
    WeldJoint,
    JacobianWrtVariable,
    PiecewisePolynomial,
    LeafSystem_,
    TemplateSystem,
    BasicVector_,
    Demultiplexer,
    Multiplexer,
    Expression,
    DirectTranscription,
    PositionCost,
    SolverOptions,
    CommonSolverOption,
    System,
    ConstantVectorSource,
)
import matplotlib.pyplot as plt

In [ ]:
meshcat = StartMeshcat()

In [ ]:
@TemplateSystem.define("DynamicActuator_")
def DynamicActuator_(T):
    class Impl(LeafSystem_[T]):
        def _construct(self, motor_inertia, torque_constant, viscous_friction, resistance, inductance, converter=None):
            LeafSystem_[T].__init__(self, converter)
            self.motor_inertia = motor_inertia 
            self.torque_constant = torque_constant
            self.back_emf_constant = torque_constant # assumes its equal to the torque constant
            self.viscous_friction = viscous_friction
            self.resistance = resistance
            self.inductance = inductance

            self.old_dtheta = 0
            self.V = self.DeclareVectorInputPort("voltage", BasicVector_[T](1))
            self.dtheta = self.DeclareVectorInputPort("dq", BasicVector_[T](1))
            #self.ddtheta = self.DeclareVectorInputPort("ddq", BasicVector_[T](1))
            #self.DeclareVectorOutputPort("torque", 1, self.CalcTorque, {self.xcdot_ticket()}) # didn't work
            self.DeclareVectorOutputPort("torque", 1, self.CalcTorque, {self.all_state_ticket()})

        def CalcTorque(self, context, output):
            V = self.V.Eval(context)
            dtheta = self.dtheta.Eval(context)
            #ddtheta = self.ddtheta.Eval(context)
            ddtheta = dtheta - self.old_dtheta
            self.old_dtheta = dtheta

            # V = R*i + L*i + k_b*omega
            current = (V - self.back_emf_constant * dtheta) / (self.resistance + self.inductance)

            # J*omega_dot = k_t*i - v*oemga + Tau_ext
            torque = self.motor_inertia * ddtheta - self.torque_constant * current + self.viscous_friction * dtheta
            output.SetFromVector([torque])

        def _construct_copy(
            self,
            other,
            converter=None,
        ):
            Impl._construct(self, other.motor_inertia, other.torque_constant, other.viscous_friction, other.resistance, other.inductance, converter=converter)

    return Impl


In [ ]:
from pydrake.all import (PlanarJoint, JointIndex, namedview)

def MakeNamedView(plant):
       names_pos = [None]*plant.num_positions()
       for ind in range(plant.num_joints()):
           joint = plant.get_joint(JointIndex(ind))
           if type(joint) == PlanarJoint:
               names_pos[joint.position_start()]     = joint.name() + "_x"
               names_pos[joint.position_start() + 1] = joint.name() + "_y"
               names_pos[joint.position_start() + 2] = joint.name() + "_theta"
           elif type(joint) != WeldJoint:
               names_pos[joint.position_start()] = joint.name() + "_p"
       for ind in plant.GetFloatingBaseBodies():
           body = plant.get_body(ind)
           start = body.floating_positions_start()
           body_name = body.name()
           names_pos[start] = body_name+'_qw'
           names_pos[start+1] = body_name+'_qx'
           names_pos[start+2] = body_name+'_qy'
           names_pos[start+3] = body_name+'_qz'
           names_pos[start+4] = body_name+'_x'
           names_pos[start+5] = body_name+'_y'
           names_pos[start+6] = body_name+'_z'
       
       names_vel = [None]*plant.num_velocities()
       for ind in range(plant.num_joints()):
           joint = plant.get_joint(JointIndex(ind))
           if type(joint) == PlanarJoint:
               names_vel[joint.velocity_start()]     = joint.name() + "_vx"
               names_vel[joint.velocity_start() + 1] = joint.name() + "_vy"
               names_vel[joint.velocity_start() + 2] = joint.name() + "_vtheta"
           elif type(joint) != WeldJoint:
               names_vel[joint.velocity_start()] = joint.name() + "_v"
       for ind in plant.GetFloatingBaseBodies():
           body = plant.get_body(ind)
           start = body.floating_velocities_start() - plant.num_positions()
           body_name = body.name()
           names_vel[start] = body_name+'_wx'
           names_vel[start+1] = body_name+'_wy'
           names_vel[start+2] = body_name+'_wz'
           names_vel[start+3] = body_name+'_vx'
           names_vel[start+4] = body_name+'_vy'
           names_vel[start+5] = body_name+'_vz'
       
       return namedview("state", names_pos + names_vel)
   

In [ ]:
class RobotArm:
    def __init__(self, time_step = 0.001, link_fractions = [0.5, 0.3, 0.2], total_link_length = 0.3):
        # Motor variables 
        J = 0.0035/18.75**2
        k_t = 0.18
        nu = 0.0005
        R = 2.0
        L = 0

        model = self._makeModel(link_fractions, total_link_length)

        builder = DiagramBuilder()
        plant, scene_graph = AddMultibodyPlantSceneGraph(builder, time_step=time_step)
        plant.mutable_gravity_field().set_gravity_vector([0,0,0])
    
        acrobot_index = Parser(plant).AddModelsFromString(file_contents=model,  file_type="urdf")

        #Weld base link 
        plant.WeldFrames(plant.world_frame(), plant.GetFrameByName("base_link"))

        # Add joint actuators and populate limits
        self.PositionLowerLimit = []
        self.PositionUpperLimit = []
        for ind in range(plant.num_joints()):
            joint = plant.get_joint(JointIndex(ind))
            if type(joint) == RevoluteJoint:
              if joint.name() == "wrist":
                print("skipping wrist motor")
                continue
              joint_actuator = plant.AddJointActuator(joint.name() + "_motor", joint) #infinite effort limit
              #joint_actuator.SetRotorInertia(J) # Doesn't work
              self.PositionLowerLimit.append(joint.position_lower_limit())
              self.PositionUpperLimit.append(joint.position_upper_limit())
        

        plant.Finalize()
        motor1 = builder.AddSystem(DynamicActuator_[None](J, k_t, nu, R, L))
        motor2 = builder.AddSystem(DynamicActuator_[None](J, k_t, nu, R, L))

        # connect dtheta's to motors
        #demux_state = builder.AddSystem(Demultiplexer(6))
        #builder.Connect(plant.get_state_output_port(), demux_state.get_input_port(0))
        #builder.Connect(demux_state.get_output_port(3), motor1.GetInputPort("dq"))
        #builder.Connect(demux_state.get_output_port(4), motor2.GetInputPort("dq"))

        #demux_acc = builder.AddSystem(Demultiplexer(3))
        #builder.Connect(plant.get_generalized_acceleration_output_port(), demux_acc.get_input_port())
        #builder.Connect(demux_acc.get_output_port(0), motor1.GetInputPort("ddq"))
        #builder.Connect(demux_acc.get_output_port(1), motor2.GetInputPort("ddq"))


        # connect motor outputs to plant
        #mux = builder.AddSystem(Multiplexer([1,1]))
        #builder.Connect(motor1.GetOutputPort("torque"), mux.get_input_port(0))
        #builder.Connect(motor2.GetOutputPort("torque"), mux.get_input_port(1))
        #builder.Connect(mux.get_output_port(), plant.get_actuation_input_port())

        # expose motor inputs
        #demux_motor_input = builder.AddSystem(Demultiplexer(2))
        #builder.Connect(demux_motor_input.get_output_port(0), motor1.get_input_port(0))
        #builder.Connect(demux_motor_input.get_output_port(1), motor2.get_input_port(0))


        input = builder.AddSystem(PassThrough(2))
        #builder.Connect(input.get_output_port(), demux_motor_input.get_input_port())
        builder.Connect(input.get_output_port(), plant.get_actuation_input_port())
        
        
        builder.ExportInput(input.get_input_port(), "input")
        builder.ExportOutput(scene_graph.get_query_output_port(), "geometry_query")

        self.X_logger = LogVectorOutput(plant.get_state_output_port(), builder)
        self.U_logger = LogVectorOutput(input.get_output_port(), builder)

        diagram = builder.Build()
        
        self.diagram_ = diagram
        self.plant_ = plant

        self.named_view_ = MakeNamedView(self.plant_)

    def _makeModel(self, link_fractions = [0.5,0.5,0.1], total_link_length = 0.3):
      if sum(link_fractions) != 1.0:
        raise Exception("Link length fractions must sum up to 1")
      self.link_length = [total_link_length * fraction for fraction in link_fractions]
      radius = 0.02
      mass = [2 * fraction for fraction in link_fractions]
      return f"""<robot name="Acrobot">
        <link name="base_link">
          <visual>
            <geometry>
              <box size="0.01 0.01 0.01"/>
            </geometry>
            <material name="none">
              <color rgba="0 1 0 0"/>
            </material>
          </visual>
        </link>
        <link name="first_link">
          <inertial>
            <origin xyz="0 0 -{self.link_length[0]/2}" rpy="0 0 0"/>
            <mass value="{mass[0]}"/>
            <inertia ixx="1" ixy="0" ixz="0" iyy="{1/12*mass[0]*self.link_length[0]**2 + 1/4*mass[0]*radius**2}" iyz="0" izz="1"/>
          </inertial>
          <visual>
            <origin xyz="0 0 -{self.link_length[0]/2}" rpy="0 0 0"/>
            <geometry>
              <cylinder length="{self.link_length[0]}" radius="0.005"/>
            </geometry>
            <material name="red">
              <color rgba="1 0 0 1"/>
            </material>
          </visual>
        </link>
        <link name="second_link">
          <inertial>
            <origin xyz="0 0 -{self.link_length[1]/2}" rpy="0 0 0"/>
            <mass value="{mass[1]}"/>
                       <inertia ixx="1" ixy="0" ixz="0" iyy="{1/12*mass[1]*self.link_length[1]**2 + 1/4*mass[1]*radius**2}" iyz="0" izz="1"/>
          </inertial>
          <visual>
            <origin xyz="0 0 -{self.link_length[1]/2}" rpy="0 0 0"/>
            <geometry>
              <cylinder length="{self.link_length[1]}" radius=".005"/>
            </geometry>
            <material name="blue">
              <color rgba="0 0 1 1"/>
            </material>
          </visual>
        </link>
        <link name="third_link">
          <inertial>
            <origin xyz="0 0 -{self.link_length[2]/2}" rpy="0 0 0"/>
            <mass value="{mass[2]}"/>
                        <inertia ixx="1" ixy="0" ixz="0" iyy="{1/12*mass[2]*self.link_length[2]**2 + 1/4*mass[2]*radius**2}" iyz="0" izz="1"/>
          </inertial>
          <visual>
            <origin xyz="0 0 -{self.link_length[2]/2}" rpy="0 0 0"/>
            <geometry>
              <cylinder length="{self.link_length[2]}" radius=".005"/>
            </geometry>
            <material name="green">
              <color rgba="1 0 0 1"/>
            </material>
          </visual>
        </link>
        <joint name="shoulder" type="revolute">
          <parent link="base_link"/>
          <child link="first_link"/>
          <origin xyz="0 0 0"/>
          <axis xyz="0 1 0"/>
          <limit
            lower="-3.14"
            upper="3.14"/>
        </joint>
        <joint name="elbow" type="revolute">
          <parent link="first_link"/>
          <child link="second_link"/>
          <origin xyz="0 0 -{self.link_length[0]}"/>
          <axis xyz="0 1 0"/>
          <limit
            lower="-1.57"
            upper="1.57"/>
        </joint>
        <joint name="wrist" type="revolute">
          <parent link="second_link"/>
          <child link="third_link"/>
          <origin xyz="0 0 -{self.link_length[1]}"/>
          <axis xyz="0 1 0"/>
          <limit
            lower="-1.57"
            upper="1.57"/>
        </joint>
        <frame name="hand" link="third_link" xyz="0 0 -{self.link_length[2]}" rpy="0 0 0"/>
      </robot>"""
    def GetDiagram(self):
        return self.diagram_
    
    def GetPlant(self):
        return self.plant_

    def GetLogs(self, context):
        return self.StateView(self.X_logger.FindLog(context).data()), self.U_logger.FindLog(context).data()

    def CreateDefaultContext(self):
        return self.diagram_.CreateDefaultContext()
      
    
    def StateView(self, q):
        return self.named_view_(q)
    
    def init(self, context, angles = [np.pi/2, np.pi/2, np.pi/2]):
        """
        """
        self.plant_.GetJointByName("shoulder").set_angle(context, angles[0])
        self.plant_.GetJointByName("elbow").set_angle(context, angles[1])
        self.plant_.GetJointByName("wrist").set_angle(context, angles[2])


In [ ]:
builder = DiagramBuilder()

arm = RobotArm()

arm_plant = arm.GetPlant()

arm_system = builder.AddSystem(arm.GetDiagram())

visualizer = MeshcatVisualizer.AddToBuilder(builder, arm_system.GetOutputPort("geometry_query"), meshcat)

meshcat.Delete()

#X_logger = LogVectorOutput(arm_system.GetOutputPort("state"), builder)

diagram = builder.Build()
simulator = Simulator(diagram)

sim_context = simulator.get_mutable_context()
plant_context = arm_plant.GetMyContextFromRoot(sim_context)

#arm_plant.get_actuation_input_port().FixValue(plant_context, np.zeros(3))
arm_plant.get_actuation_input_port().FixValue(plant_context, np.zeros(arm_plant.num_actuators()))

#initial condition
arm.init(plant_context, [-1,0,0])


q0 = np.hstack([arm_plant.GetPositions(plant_context),  arm_plant.GetVelocities(plant_context)])
q0 = arm.StateView(q0)


#simulator.set_target_realtime_rate(1.0)
animator = visualizer.StartRecording()
#for t in np.hstack((np.arange(0, 3, 1.0/32.0))):
    #q = np.hstack([arm_plant.GetPositions(plant_context),  arm_plant.GetVelocities(plant_context)])
    #frame = animator.frame(sim_context.get_time())
    #sq = arm.StateView(q)

    #simulator.AdvanceTo(t)
    #diagram.Publish(sim_context)

simulator.AdvanceTo(3.0)
visualizer.PublishRecording()

qf = np.hstack([arm_plant.GetPositions(plant_context),  arm_plant.GetVelocities(plant_context)])
qf = arm.StateView(qf)

#x_log = arm.StateView(X_logger.FindLog(sim_context).data())
#x_log = X_logger.FindLog(sim_context).data()

#fig, ax = plt.subplots(figsize=(14,6))
#ax.plot(x_log.shoulder_p)
#ax.plot(x_log[0,:])
#ax.plot(x_log[1,:])
#ax.plot(x_log[2,:])
#ax.plot(x_log[3,:])
#ax.plot(x_log[4,:])
#ax.plot(x_log[5,:])
#ax.plot(x_log[6,:])
#ax.plot(x_log[7,:])
#ax.plot(x_log[8,:])


In [ ]:
def visualize_trajectory(x_traj, u_traj = None):
    builder = DiagramBuilder()
    arm = RobotArm(0.01)

    arm_plant = arm.GetPlant()

    arm_system = builder.AddSystem(arm.GetDiagram())
    visualizer = MeshcatVisualizer.AddToBuilder(builder, arm_system.GetOutputPort("geometry_query"), meshcat)

    meshcat.Delete()
    diagram = builder.Build()
    context = diagram.CreateDefaultContext()
    animator = visualizer.StartRecording()

    U_logs = []
    for t in np.hstack((np.arange(x_traj.start_time(), x_traj.end_time(), 1.0/60.0), x_traj.end_time())):
        context.SetTime(t)
        q = x_traj.value(t)
        if u_traj:
            U_logs.append(u_traj.value(t))
        context.SetDiscreteState(q)
        frame = animator.frame(context.get_time())
        diagram.Publish(context)
    
    visualizer.StopRecording()
    visualizer.PublishRecording()

    return visualizer, arm.GetLogs(context)[0], np.hstack(U_logs)

# Trajectory Optimization

In [ ]:
options = SolverOptions()
options.SetOption(CommonSolverOption.kPrintFileName, "./debugOutput.txt")
snopt= SnoptSolver().solver_id()
options.SetOption(snopt, 'Iterations Limits', 1e6) # 1e5
options.SetOption(snopt, 'Major Feasibility Tolerance', 1e-5) #1e-6
options.SetOption(snopt, 'Function Precision', 1e-6) 
options.SetOption(snopt, 'Major Optimality Tolerance', np.sqrt(1e-6)) #2e-6
options.SetOption(snopt, 'Major Iterations Limit', 500) 
#options.SetOption(snopt, 'Superbasics limit', 2000) #209

In [ ]:
builder = DiagramBuilder()
arm = RobotArm(0)
arm_system = builder.AddSystem(arm.GetDiagram())
arm_plant = arm.GetPlant()

arm_plant_autodiff = System.ToAutoDiffXd(arm_plant)

visualizer = MeshcatVisualizer.AddToBuilder(builder, arm_system.GetOutputPort("geometry_query"), meshcat)
meshcat.Delete()

diagram = builder.Build()
diagram_autodiff = System.ToAutoDiffXd(diagram)

context = diagram.CreateDefaultContext()
context_autodiff = diagram_autodiff.CreateDefaultContext()

diagram.Publish(context)
diagram_autodiff.Publish(context_autodiff)

plant_context = arm_plant.GetMyContextFromRoot(context)
#plant_context_autodiff = arm_plant_autodiff.GetMyContextFromRoot(context_autodiff)
#plant_context_autodiff = arm_plant_autodiff.GetMyContextFromRoot(context_autodiff)
plant_context_autodiff = arm_plant_autodiff.CreateDefaultContext()

x_traj, u_traj = None, None
#duration_bounds = [0.1,3]
duration_bounds = [0.1,1]
arm.PositionLowerLimit += [-1.57]
arm.PositionUpperLimit += [1.57]
for N in [11, 21, 41, 81, 101, 121, 141, 161, 181, 201]:#, 321]:
    max_dt = 0.5
    max_tf = N * max_dt
    arm_context = arm.CreateDefaultContext()
    #arm_context = context_autodiff
    print("-----------------------------------")
    print(f"min_dt: {duration_bounds[0]/(N-1)}")
    print(f"max_dt: {duration_bounds[1]/(N-1)}")
    print(f"N: {N}")
    dircol = DirectCollocation(arm_system,
                            arm_context,
                            num_time_samples=N,
                            minimum_timestep=duration_bounds[0]/N,
                            maximum_timestep=duration_bounds[1]/N)
 
    prog = dircol.prog()
    dircol.AddEqualTimeIntervalsConstraints()
    
    actuation_limit = 2.0  # N*m.
    #actuation_limit = 12.0  # V.
    u = dircol.input()

    for i in range(len(u)):
        dircol.AddConstraintToAllKnotPoints(-actuation_limit <= u[i])
        dircol.AddConstraintToAllKnotPoints(u[i] <= actuation_limit)
    
    # intial torque is 0
    ui = dircol.input(0)
    for i in range(len(ui)):
        dircol.AddConstraintToAllKnotPoints(ui[i] == 0)
        

    # joint position constraints 
    s = dircol.state()
    dircol.AddConstraintToAllKnotPoints(s[0] <= arm.PositionUpperLimit[0])
    dircol.AddConstraintToAllKnotPoints(s[1] <= arm.PositionUpperLimit[1])
    dircol.AddConstraintToAllKnotPoints(s[2] <= arm.PositionUpperLimit[2])
    dircol.AddConstraintToAllKnotPoints(s[0] >= arm.PositionLowerLimit[0])
    dircol.AddConstraintToAllKnotPoints(s[1] >= arm.PositionLowerLimit[1])
    dircol.AddConstraintToAllKnotPoints(s[2] >= arm.PositionLowerLimit[2])
    
        # joint velocity constraints 
    max_rad_s = 250
    dircol.AddConstraintToAllKnotPoints(s[3] <= max_rad_s)
    dircol.AddConstraintToAllKnotPoints(s[4] <= max_rad_s)
    dircol.AddConstraintToAllKnotPoints(s[5] <= max_rad_s)
    dircol.AddConstraintToAllKnotPoints(s[3] >= -max_rad_s)
    dircol.AddConstraintToAllKnotPoints(s[4] >= -max_rad_s)
    dircol.AddConstraintToAllKnotPoints(s[5] >= -max_rad_s)
    
    #initial joint velocity is 0
    qi = dircol.initial_state()
    dircol.AddConstraintToAllKnotPoints(qi[3] == 0)
    dircol.AddConstraintToAllKnotPoints(qi[4] == 0)
    dircol.AddConstraintToAllKnotPoints(qi[5] == 0)

    #initial joint angle of the wrist joint is 0
    #prog.AddConstraint(qi[2] == 0)
    
    
    def velocity_cost(qv):
        hand = arm_plant_autodiff.GetFrameByName("hand")
        arm_plant_autodiff.SetPositionsAndVelocities(plant_context_autodiff, qv)
        J = arm_plant_autodiff.CalcJacobianTranslationalVelocity(plant_context_autodiff,
            JacobianWrtVariable.kQDot, hand, [0,0,0], arm_plant_autodiff.world_frame(), arm_plant_autodiff.world_frame())
        v = J.dot(qv[3:])
        v = v[[0,2]]
        return -v.dot(v)
    
    

    qf = dircol.final_state()
    #dircol.AddFinalCost(-qf[3:].dot(qf[3:]))
    #hand = arm_plant.GetFrameByName("hand")
    #cost = PositionCost(arm_plant, arm_plant.world_frame(), [0.3,0,0], hand, [0,0,0], np.eye(3), plant_context)
    
    prog.AddCost(velocity_cost, vars = qf) 
    #dircol.AddFinalCost(cost, qf)
    #dircol.AddRunningCost(ui.dot(ui))
    #prog.AddCost(ui.dot(ui))

    for i in range(N - 1):
        ui = dircol.input(i)
        ui_1 = dircol.input(i+1)
        u_stack = np.hstack([ui, ui_1])
        #prog.AddCost(lambda x: (x[2] - x[0])**2 + (x[3] - x[1])**2, u_stack)

    if not x_traj is None:
        dircol.SetInitialTrajectory(PiecewisePolynomial(), x_traj)
        #for n in range(N):
        #    prog.SetInitialGuess(dircol.state(n), initial_x_traj.value(duration_bounds[1]*n/(N-1)))
    else:
        initial_x_traj = PiecewisePolynomial.FirstOrderHold([0., duration_bounds[1]], np.hstack([np.array([-3.14,-1.57,-1.57, 0, 0,0])[:,None],
                                                            np.array([3.14,1.57,1.57, 100,100,100])[:,None]]))
        dircol.SetInitialTrajectory(PiecewisePolynomial(), initial_x_traj)
    
        for n in range(N):
            prog.SetInitialGuess(dircol.state(n), initial_x_traj.value(duration_bounds[1]*n/(N-1)))
    

    solver = SnoptSolver()

    result = solver.Solve(prog, solver_options=options)
    if not result.is_success():
        print("failed")
        break
    
    print(f"success: {result.is_success()}")
    if result.is_success():
        x_traj = dircol.ReconstructStateTrajectory(result)
        u_traj = dircol.ReconstructInputTrajectory(result)
        _, x_log, u_log = visualize_trajectory(x_traj, u_traj)

### Visualize traj opt results

In [ ]:
_, x_log, u_log = visualize_trajectory(x_traj, u_traj)

In [ ]:
def plot_traj(x_traj, u_traj):
    #t = np.linspace(x_traj.start_time(), x_traj.end_time(), x_log.shape[1])
    t = np.linspace(x_traj.start_time(), x_traj.end_time(), int(1e5))

    x_log = np.array([x_traj.value(i) for i in t]).T
    u_log = np.array([u_traj.value(i) for i in t]).T

    velocities = []
    for q in x_log.T:
        hand = arm_plant.GetFrameByName("hand")
        arm_plant.SetPositionsAndVelocities(plant_context, q)
        J = arm_plant.CalcJacobianTranslationalVelocity(plant_context,
            JacobianWrtVariable.kQDot, hand, [0,0,0], arm_plant.world_frame(), arm_plant.world_frame())
        v = J.dot(q[3:])
        velocities.append(v)
    velocities = np.array(velocities) 

    fig, ax = plt.subplots(2, 2, figsize=(15,12), facecolor='white')
    # joint angles
    ax[0,0].plot(t, x_log.T[:,0], label='q1')
    ax[0,0].plot(t, x_log.T[:,1], label='q2')
    ax[0,0].plot(t, x_log.T[:,2], label='q3')
    ax[0,0].set_title("Joint Positions")
    ax[0,0].set_xlabel("t (s)")
    ax[0,0].set_ylabel("theta (rad)")
    ax[0,0].legend()

    #joint velocities
    ax[0,1].plot(t, x_log.T[:,3], label='dq1')
    ax[0,1].plot(t, x_log.T[:,4], label='dq2')
    ax[0,1].plot(t, x_log.T[:,5], label='dq3')
    ax[0,1].set_title("Joint Velocities")
    ax[0,1].set_xlabel("t (s)")
    ax[0,1].set_ylabel("dtheta (rad/s)")
    ax[0,1].legend()

    ax[1,0].plot(t, u_log.T[:,0], label='tau1')
    ax[1,0].plot(t, u_log.T[:,1], label='tau2')
    ax[1,0].set_title("Joint Torques")
    ax[1,0].set_xlabel("t (s)")
    ax[1,0].set_ylabel("torque (Nm)")
    ax[1,0].legend()

    ax[1,1].plot(t, velocities[:,0], label='dx')
    ax[1,1].plot(t, velocities[:,2], label='dy')
    ax[1,1].plot(t, np.sqrt(velocities[:,0]**2 +  velocities[:,2]**2), label='sqrt(dx**2+dy**2)')
    ax[1,1].set_title("Endeffector velocity")
    ax[1,1].set_xlabel("t (s)")
    ax[1,1].set_ylabel("speed (m/s)")
    ax[1,1].legend()

    return fig

In [ ]:
plot_traj(x_traj, u_traj)